In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import time
import random
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



In [13]:
df = pd.read_csv('./PartMC_unlabeled.csv')
df

,Scenario_ID,DayofYear,Time(hr),O3 (ppb),CO (ppb),NO (ppb),NOx (ppb),CH3OH (ppb),C2H6 (ppb),ETH (ppb),...,Temperature(K),RH,BC (ug/m3),OA (ug/m3),NH4 (ug/m3),NO3 (ug/m3),SO4 (ug/m3),Chi_a,Chi_o,Chi_h
0,1,234,0,162.749629,564.065217,0.000020,1.915543,0.851024,0.970604,2.043904,...,294.744,0.7316,0.143005,4.862696,0.135815,0.006440,0.354422,0.966419,0.940670,0.995313
1,1,234,1,155.856708,545.475849,0.000020,1.817247,0.841861,0.972034,1.893518,...,294.744,0.7316,0.136778,4.639779,0.130955,0.006417,0.341751,0.965554,0.939991,0.994808
2,1,234,2,149.410138,527.862373,0.000019,1.747696,0.830677,0.973393,1.757071,...,294.744,0.7316,0.129698,4.405739,0.125520,0.006092,0.327533,0.966232,0.942839,0.994429
3,1,234,3,143.372736,511.173060,0.000018,1.685793,0.818304,0.974683,1.633039,...,294.744,0.7316,0.124181,4.189703,0.120432,0.005654,0.314178,0.966096,0.942376,0.994009
4,1,234,4,137.714718,495.359429,0.000017,1.628340,0.805024,0.975910,1.520084,...,294.744,0.7316,0.115353,3.889528,0.112800,0.005151,0.294241,0.966149,0.942376,0.993667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1000,205,20,106.914318,314.335501,0.211579,1.599568,0.457486,0.970724,0.112712,...,297.607,0.4218,0.916649,1.397415,0.263742,0.156934,0.620478,0.902766,0.953099,0.915871
19996,1000,205,21,108.907729,308.524698,0.180998,1.325367,0.437233,0.965068,0.091298,...,297.607,0.4218,0.872607,1.370393,0.234906,0.032324,0.640674,0.880681,0.946499,0.893130
19997,1000,205,22,110.118135,302.708175,0.154614,1.126385,0.419997,0.959102,0.074098,...,297.607,0.4218,0.820916,1.333043,0.240318,0.024840,0.659333,0.890383,0.947266,0.902580
19998,1000,205,23,110.537684,297.035474,0.136611,1.007419,0.405327,0.953550,0.061960,...,297.607,0.4218,0.777986,1.302668,0.248642,0.020875,0.680814,0.897896,0.947827,0.908951


In [6]:
y_train_np = df['Chi_a'].values
y_train_np

array([0.80448484, 0.82562121, 0.77578576, ..., 0.98146615, 0.98124413,
       0.98085214], shape=(1250,))

In [14]:
df1 = pd.read_csv('../../AL_partmc/PartMC_data/PartMC_unlabeled.csv')
df1

,Scenario_ID,DayofYear,Time(hr),O3 (ppb),CO (ppb),NO (ppb),NOx (ppb),CH3OH (ppb),C2H6 (ppb),ETH (ppb),...,PAR (ppb),OLET (ppb),Temperature(K),RH,BC (ug/m3),OA (ug/m3),NH4 (ug/m3),NO3 (ug/m3),SO4 (ug/m3),Chi
0,1,234,0.0,162.749629,564.065217,0.000020,1.915543,0.851024,0.970604,2.043904,...,70.126412,0.005952,294.744,0.7316,0.143005,4.862696,0.135815,0.006440,0.354422,0.966419
1,1,234,1.0,155.856708,545.475849,0.000020,1.817247,0.841861,0.972034,1.893518,...,66.481929,0.005475,294.744,0.7316,0.136778,4.639779,0.130955,0.006417,0.341751,0.965554
2,1,234,2.0,149.410138,527.862373,0.000019,1.747696,0.830677,0.973393,1.757071,...,63.039065,0.005164,294.744,0.7316,0.129698,4.405739,0.125520,0.006092,0.327533,0.966232
3,1,234,3.0,143.372736,511.173060,0.000018,1.685793,0.818304,0.974683,1.633039,...,59.787133,0.004976,294.744,0.7316,0.124181,4.189703,0.120432,0.005654,0.314178,0.966096
4,1,234,4.0,137.714718,495.359429,0.000017,1.628340,0.805024,0.975910,1.520084,...,56.714942,0.004875,294.744,0.7316,0.115353,3.889528,0.112800,0.005151,0.294241,0.966149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1000,205,20.0,106.914318,314.335501,0.211579,1.599568,0.457486,0.970724,0.112712,...,30.048335,0.002199,297.607,0.4218,0.916649,1.397415,0.263742,0.156934,0.620478,0.902766
19996,1000,205,21.0,108.907729,308.524698,0.180998,1.325367,0.437233,0.965068,0.091298,...,26.881377,0.001554,297.607,0.4218,0.872607,1.370393,0.234906,0.032324,0.640674,0.880681
19997,1000,205,22.0,110.118135,302.708175,0.154614,1.126385,0.419997,0.959102,0.074098,...,23.926835,0.001258,297.607,0.4218,0.820916,1.333043,0.240318,0.024840,0.659333,0.890383
19998,1000,205,23.0,110.537684,297.035474,0.136611,1.007419,0.405327,0.953550,0.061960,...,21.295807,0.001168,297.607,0.4218,0.777986,1.302668,0.248642,0.020875,0.680814,0.897896


In [16]:
(df['Chi_a']-df1['Chi']).sum()

np.float64(-3.789091757022689e-08)

In [2]:
# ---------- Reproducibility ----------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():                  
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(SEED)

# ---------- Setup ----------
ensure_dir(RESULTS_DIR)
ensure_dir(MODEL_DIR)
ensure_dir(PLOT_DIR)
logger = init_logger(LOG_PATH)
logger.info("Start Active Learning run (Logit-Normal Mode)")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")
logger.info(f"Query Strategy: {QUERY_STRATEGY} | Beta: {BETA}")

def save_dist_snapshot(round_idx, X_pool_tensor, learner, selected_idx, save_dir):
    logger.info(f"Saving uncertainty distribution for Round {round_idx}...")
    _, epi, alea = learner.model.predict_with_uncertainties(X_pool_tensor, mc_runs=50)
    
    is_selected = np.zeros(len(X_pool_tensor), dtype=bool)
    is_selected[selected_idx] = True
    
    save_path = os.path.join(save_dir, f"dist_round_{round_idx}.npz")
    np.savez(save_path, 
             epi=epi.cpu().numpy(), 
             alea=alea.cpu().numpy(), 
             is_selected=is_selected)
             
def to_logit_space(y_tensor, eps=1e-6):
    y_clamped = torch.clamp(y_tensor, min=eps, max=1.0-eps)
    z = torch.log(y_clamped / (1.0 - y_clamped))
    return z

# ---------- Data Loading & Preprocessing ----------
features = [
    'O3 (ppb)', 'CO (ppb)', 'NO (ppb)', 'NOx (ppb)', 'ETH (ppb)', 'TOL(ppb)',
    'XYL (ppb)', 'ALD2 (ppb)', 'AONE (ppb)', 'PAR (ppb)', 'OLET (ppb)',
    'Temperature(K)', 'RH', 'BC (ug/m3)', 'OA (ug/m3)', 'NH4 (ug/m3)',
    'NO3 (ug/m3)', 'SO4 (ug/m3)'
]

# Load Data
logger.info("Loading data...")
labeled_df = pd.read_csv('../PartMC_data/PartMC_labeled.csv')
unlabeled_df = pd.read_csv('../PartMC_data/PartMC_unlabeled.csv')
valid_df = pd.read_csv('../PartMC_data/PartMC_valid.csv')
test_df  = pd.read_csv('../PartMC_data/PartMC_test.csv')
scenarios_pool = unlabeled_df['Scenario_ID'].values
logger.info(f"Loaded Pool Scenarios. Unique Scenarios: {len(np.unique(scenarios_pool))}")
# Extract Values
X_train_np = labeled_df[features].values
y_train_np = labeled_df.iloc[:, 23].values

X_pool_np = unlabeled_df[features].values
y_pool_np = unlabeled_df.iloc[:, 23].values

X_valid_np = valid_df[features].values
y_valid_np = valid_df.iloc[:, 23].values
X_test_np  = test_df[features].values
y_test_np  = test_df.iloc[:, 23].values

# Scaling (Fit on initial labeled data)
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train_np)
X_pool_np  = scaler.transform(X_pool_np)
X_valid_np = scaler.transform(X_valid_np)
X_test_np  = scaler.transform(X_test_np)

# Convert to Tensors
X_train = torch.tensor(X_train_np, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train_np, dtype=torch.float32, device=device)
X_pool = torch.tensor(X_pool_np, dtype=torch.float32, device=device)
y_pool = torch.tensor(y_pool_np, dtype=torch.float32, device=device)
X_valid = torch.tensor(X_valid_np, dtype=torch.float32, device=device)
y_valid = torch.tensor(y_valid_np, dtype=torch.float32, device=device)
X_test  = torch.tensor(X_test_np,  dtype=torch.float32, device=device)
y_test  = torch.tensor(y_test_np,  dtype=torch.float32, device=device)

y_valid_orig_np = y_valid.cpu().numpy()
y_test_orig_np  = y_test.cpu().numpy()

y_train_z = to_logit_space(y_train)
y_pool_z  = to_logit_space(y_pool) 
y_valid_z = to_logit_space(y_valid)


# Learner wrapped
def get_new_learner():
    model_params = {
        'in_features': INPUT_FEATURES,
        'd_model': D_MODEL,
        'nhead': N_HEAD,
        'num_layers': NUM_LAYERS,
        'dim_feedforward': DIM_FEEDFORWARD,
        'dropout_p': DROPOUT
    }
    
    return DeepEnsembleAgent(
        model_class=GaussianFTTransformer,
        model_params=model_params,
        lr=LEARNING_RATE, 
        weight_decay=WEIGHT_DECAY,
        epochs=AL_EPOCHS, 
        batch_size=BATCH_SIZE,
        n_ensembles=N_ENSEMBLES 
    )

# ---------- Initial Training (Round 0) ----------
logger.info(f"Start Initial Training on {len(X_train)} samples...")
learner = get_new_learner()
learner.fit(X_train, y_train_z, X_val=X_valid, y_val_z=y_valid_z)

performance_log = []
queried_indices_all = []
total_start_time = time.time()

logger.info("Evaluating Initial Model (Round 0)...")
# Evaluate Initial Model
val_preds, val_unc = learner.predict(X_valid)
val_preds = val_preds.cpu().numpy()
val_unc = val_unc.cpu().numpy()
init_val_rmse = mean_squared_error(y_valid_orig_np, val_preds, squared=False)
init_val_r2   = r2_score(y_valid_orig_np, val_preds)

test_preds, test_unc = learner.predict(X_test)
test_preds = test_preds.cpu().numpy()
test_unc = test_unc.cpu().numpy()
init_test_rmse = mean_squared_error(y_test_orig_np, test_preds, squared=False)
init_test_r2   = r2_score(y_test_orig_np, test_preds)

logger.info(f"Round 0 (Initial): Val RMSE={init_val_rmse:.4f}, R2={init_val_r2:.4f}")
logger.info(f"Round 0 (Initial): Test RMSE={init_test_rmse:.4f}, R2={init_test_r2:.4f}")

performance_log.append((0, init_val_rmse, init_val_r2, init_test_rmse, init_test_r2, 0))

# ---------- Active Learning Loop ----------
for i in range(N_QUERIES):
    logger.info(f"\n=== Query Round {i+1}/{N_QUERIES} ===")

    # query using numpy pool
    n_inst = min(QUERY_BATCH_SIZE, X_pool_np.shape[0])
    
    logger.info(f"Selecting scenarios {QUERY_BATCH_SIZE} using strategy: {QUERY_STRATEGY}...")
    
    query_idx = learner.select_samples(
      X_pool, 
      k=n_inst, 
      strategy=QUERY_STRATEGY,
      mc_runs=None,
      alpha=ALPHA,   
      beta=BETA,
      group_ids=scenarios_pool
    )
    
    selected_ids = np.unique(scenarios_pool[query_idx])
    logger.info(f"Selected Scenario IDs: {selected_ids}")
    logger.info(f"Total points selected: {len(query_idx)}")
    
    ANALYSIS_DIR = os.path.join(RESULTS_DIR, 'analysis_data')
    # ensure_dir(dist_dir)
    # save_dist_snapshot(i+1, X_pool, learner, query_idx, dist_dir)
    current_y_pool_np = y_pool.cpu().numpy()
    
    # Retrieve Data
    X_new = X_pool[query_idx]
    y_new_z = y_pool_z[query_idx]
    
    X_train_temp = torch.cat([X_train, X_new], dim=0)
    save_comprehensive_analysis(
        round_idx=i+1,
        learner=learner,
        X_pool=X_pool,                  # Current Pool (before removal)
        y_pool_orig=current_y_pool_np,
        selected_idx=query_idx,         # Indices selected in this pool
        pool_scenario_ids=scenarios_pool, # Current Pool IDs
        # X_train_accumulated=X_train_temp, # Projected New Training Set
        X_test=X_test,
        y_test_orig=y_test_orig_np,
        save_dir=ANALYSIS_DIR
    )
    
    
    # Join new data with existing training data
    X_train = torch.cat([X_train, X_new], dim=0)
    y_train_z = torch.cat([y_train_z, y_new_z], dim=0)
    
    # Remove from Pool
    mask = torch.ones(len(X_pool), dtype=torch.bool, device=device)
    mask[query_idx] = False
    X_pool = X_pool[mask]
    y_pool_z = y_pool_z[mask]
    scenarios_pool = scenarios_pool[mask.cpu().numpy()]
    
    # Record indices
    queried_indices_all.extend(query_idx.tolist())
    logger.info(f"Pool size: {len(X_pool)}, Train size: {len(X_train)}")
    
    start_time = time.time()
    learner = get_new_learner()
    learner.fit(X_train, y_train_z, X_val=X_valid, y_val_z=y_valid_z)
    elapsed_time = time.time() - start_time
    logger.info(f"Training time: {elapsed_time:.2f}s")
    
    # Evaluation: Validation
    val_preds, val_unc = learner.predict(X_valid)
    val_preds = val_preds.cpu().numpy()
    val_unc = val_unc.cpu().numpy()
    rmse = mean_squared_error(y_valid_orig_np, val_preds, squared=False)
    mae  = mean_absolute_error(y_valid_orig_np, val_preds)
    r2   = r2_score(y_valid_orig_np, val_preds)

    # Evaluation: Test
    test_preds, test_unc = learner.predict(X_test)
    test_preds = test_preds.cpu().numpy()
    test_unc = test_unc.cpu().numpy()
    test_rmse = mean_squared_error(y_test_orig_np, test_preds, squared=False)
    test_mae  = mean_absolute_error(y_test_orig_np, test_preds)
    test_r2   = r2_score(y_test_orig_np, test_preds)

    logger.info(f"Round {i+1}: Val RMSE={rmse:.4f}, R2={r2:.4f}")
    logger.info(f"Round {i+1}: Test RMSE={test_rmse:.4f}, R2={test_r2:.4f}")
    performance_log.append((i+1, rmse, mae, r2, test_rmse, test_mae, test_r2, elapsed_time))
        
    # save checkpoint
    for idx, model in enumerate(learner.models):
        torch.save(
            model.state_dict(),
            f"{MODEL_DIR}/model_round_{i+1}_ensemble_{idx}.pt"
        )
    # torch.save(learner.model.state_dict(), f"{MODEL_DIR}/model_round_{i+1}.pt")

# ---------- Final Wrap-up ----------
total_elapsed_time = time.time() - total_start_time
logger.info(f"Total Active Learning time: {total_elapsed_time:.2f}s")

# Save queried indices and performance
np.save(QUERIED_IDX_PATH, np.array(queried_indices_all))
save_performance(performance_log, PERFORMANCE_PATH) # Make sure utils handles the extra columns if needed
# Plot RMSE Curve
rounds = [x[0] for x in performance_log]
rmses = [x[1] for x in performance_log]
# save_plot(rmses, f"{PLOT_DIR}/rmse_learning_curve.png", title="Validation RMSE over Queries", ylabel="RMSE")

logger.info("Finished Active Learning Run.")


2025-12-19 15:33:42,463 | Start Active Learning run (Logit-Normal Mode)
2025-12-19 15:33:42,463 | Using device: cuda
2025-12-19 15:33:42,463 | Query Strategy: cis_gating | Beta: 1.0
2025-12-19 15:33:42,464 | Loading data...
2025-12-19 15:33:42,511 | Loaded Pool Scenarios. Unique Scenarios: 800
/home/fei/miniconda3/envs/AL/lib/python3.9/site-packages/torch/cuda/__init__.py:287: UserWarning: 
NVIDIA GeForce RTX 5090 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
